In [40]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import config
import mimetypes
import tarfile
import os

# Your client credentials
client_id = config.client_id
client_secret = config.api_secret

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)


# Examples for S3OLCI

True Color


In [45]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B08", "B06", "B04"],
    output: {
      bands: 3,
      sampleType: "AUTO", // default value - scales the output values from [0,1] to [0,255].
    },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B08, 2.5 * sample.B06, 2.5 * sample.B04]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                8.3333,
                41.3149,
                9.7009,
                43.0568,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3OLCI_Exmaple1.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)


Image saved


True Color (EPSG 32632)

In [41]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B08", "B06", "B04"],
    output: {
      bands: 3,
      sampleType: "AUTO", // default value - scales the data from 0-255.
    },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B08, 2.5 * sample.B06, 2.5 * sample.B04]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3OLCI_Exmaple2.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


True Color, resolution (EPSG 32632)


In [21]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B08", "B06", "B04"],
    output: {
      bands: 3,
      sampleType: "AUTO", // default value - scales the output values from [0,1] to [0,255].
    },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B08, 2.5 * sample.B06, 2.5 * sample.B04]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    },
                    "processing": {"upsampling": "BILINEAR"},
                },
            }
        ],
    },
    "output": {
        "resx": 150,
        "resy": 150,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3OLCI_Exmaple3.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


True Color, multi-band GeoTiff


In [22]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["B04", "B06", "B08"],
        units: "REFLECTANCE", // default value
      },
    ],
    output: {
      bands: 3,
      sampleType: "UINT16", //floating point values are automatically rounded to the nearest integer by the service.
    },
  }
}

function evaluatePixel(sample) {
  // Return reflectance multiplied by 10000 as integers to save processing units.
  // To obtain reflectance values, simply divide the result pixel values by 10000.
  return [10000 * sample.B08, 10000 * sample.B06, 10000 * sample.B04]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                8.3333,
                41.3149,
                9.7009,
                43.0568,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/tiff"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3OLCI_Exmaple4.tiff"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


True color and metadata (multi-part response GeoTIFF and json)

In [23]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["B04", "B06", "B08"],
        units: "REFLECTANCE",
      },
    ],
    mosaicking: Mosaicking.SIMPLE,
    output: {
      id: "default",
      bands: 3,
      sampleType: "UINT16", //floating point values are automatically rounded to the nearest integer by the service.
    },
  }
}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {
  outputMetadata.userData = { scenes: scenes.tiles }
}

function evaluatePixel(samples) {
  // Return reflectance multiplied by 10000 as integers to save processing units.
  // To obtain reflectance values, simply divide the result pixel values by 10000.
  return [10000 * samples.B08, 10000 * samples.B06, 10000 * samples.B04]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
                8.3333,
                41.3149,
                9.7009,
                43.0568,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            },
            {
                "identifier": "userdata",
                "format": {"type": "application/json"},
            },
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "application/tar"})

# Check if the response was successful
if response.status_code == 200:
    # The response is a TAR file containing the TIFF image and JSON metadata
    tar_destination_file_name = "S3OLCI_Example5.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Now, extract the contents of the TAR file
    with tarfile.open(tar_destination_path, 'r') as tar:
        tar.extractall(path="Sentinel_3")  # Extracts to the directory 'Sentinel_3'
    print("Contents extracted to the directory 'Sentinel_3'.")

else:
    print("Error:", response.status_code, response.text)

TAR file saved as Sentinel_3/S3OLCI_Example5.tar
Contents extracted to the directory 'Sentinel_3'.


OTCI as jpeg image with bounds given as polygon

In [24]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["B10", "B11", "B12"],
      },
    ],
    output: {
      id: "default",
      bands: 3,
      sampleType: "AUTO",
    },
  }
}

// Create a new visualiser to represent data
var cm = new ColorMapVisualizer([
  [0, [0, 0, 0.5]],
  [1, [0, 0.3, 0.8]],
  [1.8, [1, 0.2, 0.2]],
  [2.5, [1, 0.9, 0]],
  [4, [0, 0.8, 0.1]],
  [4.5, [0, 0.6, 0.2]],
  [5, [1, 1, 1]],
])

function evaluatePixel(sample) {
  let OTCI = (sample.B12 - sample.B11) / (sample.B11 - sample.B10)
  return cm.process(OTCI)
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            8.80279541015625,
                            42.494377798972465,
                        ],
                        [
                            8.6956787109375,
                            42.370720143531976,
                        ],
                        [
                            8.7890625,
                            42.238685347536496,
                        ],
                        [
                            8.60504150390625,
                            42.20614200929954,
                        ],
                        [
                            8.70391845703125,
                            42.15322331239858,
                        ],
                        [
                            8.83575439453125,
                            41.97991089691236,
                        ],
                        [
                            8.81378173828125,
                            41.797935707842974,
                        ],
                        [
                            8.9208984375,
                            41.777456667491066,
                        ],
                        [
                            8.94012451171875,
                            41.68316883525891,
                        ],
                        [
                            9.0472412109375,
                            41.52297326747377,
                        ],
                        [
                            9.35760498046875,
                            41.70777900286713,
                        ],
                        [
                            9.33013916015625,
                            42.06764572379527,
                        ],
                        [
                            9.48394775390625,
                            42.261049162113856,
                        ],
                        [
                            9.47021484375,
                            42.51462626746592,
                        ],
                        [
                            9.33837890625,
                            42.62385465855651,
                        ],
                        [
                            9.1900634765625,
                            42.6844544397102,
                        ],
                        [
                            8.80279541015625,
                            42.494377798972465,
                        ],
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {
                    "type": "image/jpeg",
                    "quality": 90,
                },
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3OLCI_Exmaple6.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


OTCI image and value (multi-part response png and GeoTIFF containing floats)

In [25]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["B10", "B11", "B12"],
      },
    ],
    output: [
      {
        id: "default",
        bands: 1,
        sampleType: "FLOAT32",
      },
      {
        id: "otci_image",
        bands: 3,
        sampleType: "AUTO",
      },
    ],
  }
}

// Create a new visualiser to represent data
var cm = new ColorMapVisualizer([
  [0, [0, 0, 0.5]],
  [1, [0, 0.3, 0.8]],
  [1.8, [1, 0.2, 0.2]],
  [2.5, [1, 0.9, 0]],
  [4, [0, 0.8, 0.1]],
  [4.5, [0, 0.6, 0.2]],
  [5, [1, 1, 1]],
])

function evaluatePixel(sample) {
  let OTCI = (sample.B12 - sample.B11) / (sample.B11 - sample.B10)
  return {
    default: [OTCI],
    otci_image: cm.process(OTCI),
  }
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            8.80279541015625,
                            42.494377798972465,
                        ],
                        [
                            8.6956787109375,
                            42.370720143531976,
                        ],
                        [
                            8.7890625,
                            42.238685347536496,
                        ],
                        [
                            8.60504150390625,
                            42.20614200929954,
                        ],
                        [
                            8.70391845703125,
                            42.15322331239858,
                        ],
                        [
                            8.83575439453125,
                            41.97991089691236,
                        ],
                        [
                            8.81378173828125,
                            41.797935707842974,
                        ],
                        [
                            8.9208984375,
                            41.777456667491066,
                        ],
                        [
                            8.94012451171875,
                            41.68316883525891,
                        ],
                        [
                            9.0472412109375,
                            41.52297326747377,
                        ],
                        [
                            9.35760498046875,
                            41.70777900286713,
                        ],
                        [
                            9.33013916015625,
                            42.06764572379527,
                        ],
                        [
                            9.48394775390625,
                            42.261049162113856,
                        ],
                        [
                            9.47021484375,
                            42.51462626746592,
                        ],
                        [
                            9.33837890625,
                            42.62385465855651,
                        ],
                        [
                            9.1900634765625,
                            42.6844544397102,
                        ],
                        [
                            8.80279541015625,
                            42.494377798972465,
                        ],
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "otci_image",
                "format": {"type": "image/png"},
            },
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            },
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "application/tar"})

# Check if the response was successful
if response.status_code == 200:
    # The response is a TAR file containing the PNG and TIFF images
    tar_destination_file_name = "S3OLCI_Example7.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Now, extract the contents of the TAR file
    with tarfile.open(tar_destination_path, 'r') as tar:
        tar.extractall(path="Sentinel_3")  # Extracts to the directory 'Sentinel_3'
    print("Contents extracted to the directory 'Sentinel_3'.")

else:
    print("Error:", response.status_code, response.text)

TAR file saved as Sentinel_3/S3OLCI_Example7.tar
Contents extracted to the directory 'Sentinel_3'.


All S3OLCI reflectance bands as a GeoTIFF (EPSG 32632)

In [26]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: [
          "B01",
          "B02",
          "B03",
          "B04",
          "B05",
          "B06",
          "B07",
          "B08",
          "B09",
          "B10",
          "B11",
          "B12",
          "B13",
          "B14",
          "B15",
          "B16",
          "B17",
          "B18",
          "B19",
          "B20",
          "B21",
        ],
        units: "REFLECTANCE",
      },
    ],
    output: {
      bands: 21,
      sampleType: "UINT16", //floating point values are automatically rounded to the nearest integer by the service.
    },
  }
}

function evaluatePixel(sample) {
  // Return reflectance multiplied by 10000 as integers to save processing units.
  // To obtain reflectance values, simply divide the result pixel values by 10000.
  return [
    10000 * sample.B01,
    10000 * sample.B02,
    10000 * sample.B03,
    10000 * sample.B04,
    10000 * sample.B05,
    10000 * sample.B06,
    10000 * sample.B07,
    10000 * sample.B08,
    10000 * sample.B09,
    10000 * sample.B10,
    10000 * sample.B11,
    10000 * sample.B12,
    10000 * sample.B13,
    10000 * sample.B14,
    10000 * sample.B15,
    10000 * sample.B16,
    10000 * sample.B17,
    10000 * sample.B18,
    10000 * sample.B19,
    10000 * sample.B20,
    10000 * sample.B21,
  ]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-olci",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-04-04T00:00:00Z",
                        "to": "2020-04-05T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "resx": 300,
        "resy": 300,
        "responses": [{"format": {"type": "image/tiff"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

# Check if the response was successful
if response.status_code == 200:
    content_type = response.headers.get('Content-Type')
    print(f"Content-Type received: {content_type}")

    # Determine the extension based on the Content-Type
    extension = mimetypes.guess_extension(content_type)
    print(f"Guessed file extension: {extension}")

    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)

    # Set the destination file name based on the Content-Type
    destination_file_name = f"S3OLCI_Example8{extension}"
    destination_path = os.path.join("Sentinel_3", destination_file_name)

    # Save the file
    with open(destination_path, 'wb') as file:
        file.write(response.content)
    print(f"File saved as {destination_path}")

    # If the content type is 'application/tar', extract it
    if content_type == 'application/tar':
        import tarfile
        with tarfile.open(destination_path, 'r') as tar:
            tar.extractall(path="Sentinel_3")
        print("Contents extracted to the directory 'Sentinel_3'.")
    else:
        print("No extraction needed, file saved as is.")

else:
    print("Error:", response.status_code, response.text)

Content-Type received: image/tiff
Guessed file extension: .tiff
File saved as Sentinel_3/S3OLCI_Example8.tiff
No extraction needed, file saved as is.


# Examples for S3SLSTR

False Color

In [27]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["S3", "S2", "S1"],
    output: {
      bands: 3,
      sampleType: "AUTO",
    },
  }
}

function evaluatePixel(sample) {
  return [2 * sample.S3, 2 * sample.S2, 2 * sample.S1]
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                8.558382,
                41.359678,
                9.579525,
                43.055688,
            ],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3SLSTR_Exmaple1.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


False Color (EPSG 32632)

In [46]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["S3", "S2", "S1"],
    output: {
      bands: 3,
      sampleType: "AUTO",
    },
  }
}

function evaluatePixel(sample) {
  return [2 * sample.S3, 2 * sample.S2, 2 * sample.S1]
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3SLSTR_Exmaple2.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


False Color, resolution (EPSG 32632)

In [29]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["S3", "S2", "S1"],
    output: {
      bands: 3,
      sampleType: "AUTO",
    },
  }
}

function evaluatePixel(sample) {
  return [2 * sample.S3, 2 * sample.S2, 2 * sample.S1]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "resx": 250,
        "resy": 250,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3SLSTR_Exmaple3.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


Thermal IR fire emission band, gradient visualizer (K)

In [30]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["F1"],
    output: {
      bands: 3,
    },
  }
}

// Create a Red gradient visualiser from 274-450 K
var viz = ColorGradientVisualizer.createRedTemperature(274, 450)

function evaluatePixel(sample) {
  return viz.process(sample.F1)
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                -120.141,
                37.5282,
                -119.4131,
                37.8716,
            ],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2018-08-06T00:00:00Z",
                        "to": "2018-08-06T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/png"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3SLSTR_Exmaple4.png"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


False Color and metadata (multi-part GeoTIFF and json)

In [31]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["S3", "S2", "S1"],
    output: {
      id: "default",
      bands: 3,
      sampleType: "INT16", //floating point values are automatically rounded to the nearest integer by the service.
    },
    mosaicking: "TILE",
  }
}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {
  outputMetadata.userData = { scenes: scenes.tiles }
}

function evaluatePixel(sample) {
  // Return reflectance multiplied by 10000 as integers to save processing units.
  // To obtain reflectance values, simply divide the result pixel values by 10000.
  return [sample[0].S3 * 10000, sample[0].S2 * 10000, sample[0].S1 * 10000] //the values are multiplied by 10000 because output sampleType is UINT16
}
"""

request = {
    "input": {
        "bounds": {
            "bbox": [
                8.558382,
                41.359678,
                9.579525,
                43.055688,
            ],
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            },
            {
                "identifier": "userdata",
                "format": {"type": "application/json"},
            },
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "application/tar"})

# Check if the response was successful
if response.status_code == 200:
    # The response is a TAR file containing the TIFF image and possibly other files
    tar_destination_file_name = "S3SLSTR_Example5.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Now, extract the contents of the TAR file
    with tarfile.open(tar_destination_path, 'r') as tar:
        tar.extractall(path="Sentinel_3")  # Extracts to the directory 'Sentinel_3'
    print("Contents extracted to the directory 'Sentinel_3'.")

else:
    print("Error:", response.status_code, response.text)

TAR file saved as Sentinel_3/S3SLSTR_Example5.tar
Contents extracted to the directory 'Sentinel_3'.


NDVI as jpeg image with bouds given as polygon

In [32]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["S2", "S3"],
    output: {
      bands: 3,
    },
  }
}

function evaluatePixel(sample) {
  let NDVI = index(sample.S3, sample.S2)
  const viz = ColorGradientVisualizer.createWhiteGreen(-0.1, 1.0)
  return viz.process(NDVI)
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            535547.7290897672,
                            4767538.771691742,
                        ],
                        [
                            542559.6872296461,
                            4744749.907737136,
                        ],
                        [
                            550448.1401370098,
                            4660606.41005859,
                        ],
                        [
                            521523.8128100095,
                            4570327.449007649,
                        ],
                        [
                            474193.0953658272,
                            4600128.271102134,
                        ],
                        [
                            461045.67385355436,
                            4630805.5879641045,
                        ],
                        [
                            453157.22094619065,
                            4698295.685060439,
                        ],
                        [
                            497858.45408791833,
                            4741243.928667196,
                        ],
                        [
                            520647.3180425246,
                            4744749.907737136,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            535547.7290897672,
                            4767538.771691742,
                        ],
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [{"format": {"type": "image/jpeg"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # Write the binary content of the response to a file
    destination_file_name = "Sentinel_3/S3SLSTR_Exmaple6.jpeg"
    with open(destination_file_name, 'wb') as file:
        file.write(response.content)
    print("Image saved")  # Corrected the file extension in the message
else:
    print("Error:", response.status_code, response.text)

Image saved


NDVI image and value (multi-part response png and GeoTIFF)

In [33]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["S2", "S3"],
      },
    ],
    output: [
      {
        id: "default",
        bands: 1,
        sampleType: "INT16",
      },
      {
        id: "ndvi_image",
        bands: 3,
        sampleType: "AUTO",
      },
    ],
  }
}

function evaluatePixel(sample) {
  let NDVI = index(sample.S3, sample.S2)
  const viz = ColorGradientVisualizer.createWhiteGreen(-0.1, 1.0)
  return {
    default: [NDVI * 10000],
    ndvi_image: viz.process(NDVI),
  }
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            535547.7290897672,
                            4767538.771691742,
                        ],
                        [
                            542559.6872296461,
                            4744749.907737136,
                        ],
                        [
                            550448.1401370098,
                            4660606.41005859,
                        ],
                        [
                            521523.8128100095,
                            4570327.449007649,
                        ],
                        [
                            474193.0953658272,
                            4600128.271102134,
                        ],
                        [
                            461045.67385355436,
                            4630805.5879641045,
                        ],
                        [
                            453157.22094619065,
                            4698295.685060439,
                        ],
                        [
                            497858.45408791833,
                            4741243.928667196,
                        ],
                        [
                            520647.3180425246,
                            4744749.907737136,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            525906.2866474338,
                            4771044.750761681,
                        ],
                        [
                            535547.7290897672,
                            4767538.771691742,
                        ],
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
        "responses": [
            {
                "identifier": "ndvi_image",
                "format": {"type": "image/png"},
            },
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            },
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "application/tar"})

# Check if the response was successful
if response.status_code == 200:
    # The response is a TAR file containing the requested images
    tar_destination_file_name = "S3SLSTR_Example7.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Now, extract the contents of the TAR file
    with tarfile.open(tar_destination_path, 'r') as tar:
        tar.extractall(path="Sentinel_3")  # Extracts to the directory 'Sentinel_3'
    print("Contents extracted to the directory 'Sentinel_3'.")

else:
    print("Error:", response.status_code, response.text)

TAR file saved as Sentinel_3/S3SLSTR_Example7.tar
Contents extracted to the directory 'Sentinel_3'.


VNIR and SWIR bands as a GeoTIFF (EPSG 32632)

In [34]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["S1", "S2", "S3", "S4", "S5", "S6"],
        units: "REFLECTANCE",
      },
    ],
    output: {
      bands: 6,
      sampleType: "UINT16", //floating point values are automatically rounded to the nearest integer by the service.
    },
  }
}

function evaluatePixel(sample) {
  // Return reflectance multiplied by 10000 as integers to save processing units.
  // To obtain reflectance or BT values, simply divide the resulting pixel values by 10000.
  return [
    10000 * sample.S1,
    10000 * sample.S2,
    10000 * sample.S3,
    10000 * sample.S4,
    10000 * sample.S5,
    10000 * sample.S6,
  ]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "resx": 500,
        "resy": 500,
        "responses": [{"format": {"type": "image/tiff"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
headers = {
    "Accept": "application/tar"
}

# Send the request
response = oauth.post(url, json=request, headers=headers)

# Check if the response was successful
if response.status_code == 200:
    # Check the Content-Type of the response
    print("Content-Type:", response.headers.get('Content-Type'))

    # The response is expected to be a TAR file containing the requested images
    tar_destination_file_name = "S3SLSTR_Example8.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Attempt to open the TAR file
    try:
        with tarfile.open(tar_destination_path, 'r:*') as tar:  # 'r:*' for auto-detecting the compression
            tar.extractall(path="Sentinel_3")
        print("Contents extracted to the directory 'Sentinel_3'.")
    except tarfile.ReadError as e:
        print("Failed to open the TAR file. It might not be a valid TAR archive.")
        # Save the response content to a text file for inspection
        with open('response_content.txt', 'wb') as f:
            f.write(response.content)
        print("Response content saved to 'response_content.txt'. Please inspect this file to understand the issue.")

else:
    print("Error:", response.status_code, response.text)

# Check if the response was successful
if response.status_code == 200:
    # The response is a TAR file containing the requested images
    tar_destination_file_name = "S3SLSTR_Example8.tar"
    
    # Ensure the directory exists
    os.makedirs("Sentinel_3", exist_ok=True)
    
    # Full path to the tar file
    tar_destination_path = os.path.join("Sentinel_3", tar_destination_file_name)
    
    # Save the TAR file
    with open(tar_destination_path, 'wb') as tar_file:
        tar_file.write(response.content)
    print(f"TAR file saved as {tar_destination_path}")

    # Now, extract the contents of the TAR file
    with tarfile.open(tar_destination_path, 'r') as tar:
        tar.extractall(path="Sentinel_3")  # Extracts to the directory 'Sentinel_3'
    print("Contents extracted to the directory 'Sentinel_3'.")

else:
    print("Error:", response.status_code, response.text)

Error: 500 {"error":{"status":500,"reason":"Internal Server Error","message":"Something went wrong!","code":"RENDERER_EXCEPTION"}}
Error: 500 {"error":{"status":500,"reason":"Internal Server Error","message":"Something went wrong!","code":"RENDERER_EXCEPTION"}}


TIR bands as a GeoTIFF (EPSG 32632)

In [35]:
evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["S7", "S8", "S9", "F1", "F2"],
      },
    ],
    output: {
      bands: 5,
      sampleType: "UINT16",
    },
  }
}

function multiplyband(sample) {
  // Multiply by 100
  return 100 * sample
}

function evaluatePixel(sample) {
  // Return the bands multiplied by 100 as integers to save processing units.
  // To obtain reflectance or BT values, simply divide the resulting pixel values by 100.
  return [
    multiplyband(sample.S7),
    multiplyband(sample.S8),
    multiplyband(sample.S9),
    multiplyband(sample.F1),
    multiplyband(sample.F2),
  ]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/32632"},
            "bbox": [
                444170,
                4574059,
                557052,
                4767386,
            ],
        },
        "data": [
            {
                "type": "sentinel-3-slstr",
                "dataFilter": {
                    "timeRange": {
                        "from": "2020-06-20T00:00:00Z",
                        "to": "2020-06-20T23:59:59Z",
                    },
                    "orbitDirection": "DESCENDING",
                },
            }
        ],
    },
    "output": {
        "resx": 500,
        "resy": 500,
        "responses": [{"format": {"type": "image/tiff"}}],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

# Check if the response was successful
if response.status_code == 200:
    # The response is expected to be a TIFF file containing the requested image
    tiff_destination_file_name = "S3SLSTR_Example9.tif"
    
    # Ensure the directory exists
    directory = "Sentinel_3"
    os.makedirs(directory, exist_ok=True)
    
    # Full path to the TIFF file
    tiff_destination_path = os.path.join(directory, tiff_destination_file_name)
    
    # Save the TIFF file
    with open(tiff_destination_path, 'wb') as tiff_file:
        tiff_file.write(response.content)
    print(f"TIFF file saved as {tiff_destination_path}")
else:
    print("Error:", response.status_code, response.text)

TIFF file saved as Sentinel_3/S3SLSTR_Example9.tif
